## Normalización y tratamiento de outliers

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn import linear_model 
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot as plt
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.preprocessing import scale, StandardScaler, OneHotEncoder, MinMaxScaler
import sklearn
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer

In [94]:
df = pd.read_csv("data_2.csv")
df.head(5)

,esg_score21,esg_class21,environ_score21,environ_class21,social_score21,social_class21,govern_score21,govern_class21,turnover21,country,...,esg_exe_remuneration,esg_respect_humanr,esg_human_rights,esg_respect_freedom,esg_non_discrimination,esg_physical_risk,esg_energy_transition,esg_carbon_emission1,esg_carbon_emission2,esg_carbon_emission1_2
0,20.0,Weak performance,0.0,Weak performance,17.0,Weak performance,36.0,Limited performance,490.49,USA,...,28.0,22.0,25.0,22.0,27.0,0.0,0.0,5753.0,7903.0,5537.0
1,26.0,Weak performance,18.0,Weak performance,22.0,Weak performance,40.0,Limited performance,2122.24,USA,...,43.0,31.0,28.0,23.0,33.0,43.0,13.0,2300.0,40984.0,43284.0
2,31.0,Limited performance,20.0,Weak performance,29.0,Weak performance,48.0,Limited performance,102.73,USA,...,46.0,46.0,34.0,24.0,47.0,39.0,10.0,748.0,7379.0,8128.0
3,24.0,Weak performance,9.0,Weak performance,21.0,Weak performance,38.0,Limited performance,54.52,USA,...,29.0,36.0,27.0,23.0,35.0,1.0,5.0,6222.0,18748.0,24970.0
4,24.0,Weak performance,15.0,Weak performance,19.0,Weak performance,39.0,Limited performance,0.08,USA,...,38.0,32.0,25.0,18.0,30.0,1.0,11.0,326.0,1363.0,1689.0


In [54]:
# # Paso 1: Definir las columnas específicas
# columnas_miles_usd = [
#     'turnover21', 'ebt21', 'net_income21', 'cash_flow21', 'total_assets21', 'equity21',
#     'gross_profit', 'no_current_ liabilities21', 'ono_current_liabilities21',
#     'o_circu_liabilities21', 'liabilities21', 'inmo_inmat21', 'ebitda21']

# columnas_decimales = ['margen_ebitda21', 'debt_ratio21', 'esg_size21']

# # Paso 2: Seleccionar todas las columnas numéricas excepto las anteriores
# todas_columnas_numericas = df.select_dtypes(include=['int64', 'float64']).columns
# columnas_restantes = [col for col in todas_columnas_numericas 
#                       if col not in columnas_miles_usd + columnas_decimales]

Transformación Logarítmica y Escalado Z-Score:
Aplico np.log1p para evitar valores negativos y StandardScaler para normalizar las variables en USD.

Escalado Z-Score para Variables de Puntaje:
Las variables relacionadas con puntajes ESG las normalizo directamente con StandardScaler.

Escalado StandardScaler para Variables No en USD:
Estandarizo las columnas como no_current_ liabilities21 y similares.

Transformación Logarítmica y Escalado para Variables Enteras:
Aplico log-transformación y Z-score para variables enteras.

Escalado Min-Max para Ratios:
Normalizo las variables de ratios para que queden en un rango entre 0 y 1.

Tratamiento de Emisiones de Carbono:
Log-transformación y Z-score, las aplico para reducir la asimetría de estas variables.

Tratamiento de Outliers:
Uso límites basados en el rango intercuartil (IQR) para ajustar valores extremos.

Luego, se aplico StandardScaler.

In [82]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

def transformar_dataframe(df):
    """
    Aplica transformaciones al DataFrame, incluyendo tratamiento de outliers y escalado.

    Args:
        df (pd.DataFrame): El DataFrame a transformar.

    Returns:
        pd.DataFrame: El DataFrame transformado.
    """
    def log_transform_with_sign(col):
        """Aplica logaritmo ajustado para valores positivos y negativos."""
        return np.sign(col) * np.log1p(np.abs(col))

    def tratar_outliers(df, variables):
        """Aplica clipping para tratar outliers usando el método IQR."""
        for col in variables:
            if col in df.columns:
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                df[col] = df[col].clip(lower_bound, upper_bound)
        return df

    # 1. Variables en USD: Tratamiento de outliers, log-transformación y Z-score
    variables_usd = ['turnover21', 'ebt21', 'net_income21', 'cash_flow21',
                     'total_assets21', 'equity21', 'gross_profit', 'liabilities21', 'ebitda21']
    df = tratar_outliers(df, variables_usd)
    for col in variables_usd:
        if col in df.columns:
            df[col] = StandardScaler().fit_transform(log_transform_with_sign(df[col]).values.reshape(-1, 1))

    # 2. Escalado Z-score para variables de puntaje (tratar outliers primero)
    variables_scores = ['esg_score21', 'environ_score21', 'social_score21', 'govern_score21',
                        'hrscore21', 'envscore21', 'businesscore21', 'governscore21', 
                        'cominvolvscore21', 'humanriscore21', 'esg_labour_relations',
                        'esg_personnel_manage', 'esg_responsible_manage', 'esg_career_manage',
                        'esg_labour', 'esg_improv_health', 'esg_respect_manage', 'esg_respect_environ',
                        'esg_strategy', 'esg_polution', 'esg_dev_green', 'esg_protect_bio',
                        'esg_impact_environ', 'esg_protect_water', 'esg_min_impact_energy',
                        'esg_manage_emissions', 'esg_waste_manage', 'esg_manage_polution',
                        'esg_business_travel', 'esg_manage_impact', 'esg_relations_customer',
                        'esg_product_safety', 'esg_info_customer', 'esg_respon_customer',
                        'esg_supply_chain', 'esg_sustainable_relations', 'esg_integrat_environ_factors',
                        'esg_integrat_social_factors', 'esg_irregular_practice', 'esg_prevent_corruption',
                        'esg_prevent_antcompe', 'esg_transp_integrity', 'esg_prom_social_economic',
                        'esg_impact_society', 'esg_societal_impacts', 'esg_philantropy', 'esg_bod',
                        'esg_audit_controls', 'esg_shareholders', 'esg_exe_remuneration', 'esg_respect_humanr',
                        'esg_human_rights', 'esg_respect_freedom', 'esg_non_discrimination', 'esg_physical_risk',
                        'esg_energy_transition']
    df = tratar_outliers(df, variables_scores)
    df[variables_scores] = StandardScaler().fit_transform(df[variables_scores])

    # 3. Variables no en USD: Tratamiento de outliers y transformación
    non_usd_columns = ['no_current_ liabilities21', 'ono_current_liabilities21', 'o_circu_liabilities21', 'inmo_inmat21']
    df = tratar_outliers(df, non_usd_columns)
    for col in non_usd_columns:
        if col in df.columns:
            df[col] = StandardScaler().fit_transform(log_transform_with_sign(df[col]).values.reshape(-1, 1))

    # 4. Variables enteras: Tratamiento de outliers, log-transformación y Z-score
    variables_enteras = ['firm_age', 'employees21', 'board_members', 'investees',
                         'publications', 'live_publications', 'pending_publications', 'granted_publications']
    df = tratar_outliers(df, variables_enteras)
    for col in variables_enteras:
        if col in df.columns:
            df[col] = StandardScaler().fit_transform(log_transform_with_sign(df[col]).values.reshape(-1, 1))

    # 5. Escalado Min-Max para variables de ratios (tratar outliers primero)
    variables_ratios = ['margen_ebitda21', 'margen_profit', 'debt_ratio21', 'current_ratio21',
                        'solvency21', 'liquidity_ratio21', 'liquidity_ratio_shareholders21', 'gross_margin21']
    df = tratar_outliers(df, variables_ratios)
    df[variables_ratios] = MinMaxScaler().fit_transform(df[variables_ratios])

    # 6. Emisiones de carbono: Tratamiento de outliers y transformación
    carbon_vars = ['esg_carbon_emission1', 'esg_carbon_emission2', 'esg_carbon_emission1_2']
    df = tratar_outliers(df, carbon_vars)
    for col in carbon_vars:
        if col in df.columns:
            df[col] = StandardScaler().fit_transform(log_transform_with_sign(df[col]).values.reshape(-1, 1))

    # 7. Variables financieras con tratamiento de outliers
    vars_with_outliers = ['cf_to_op_income21', 'roa_ebit21', 'roa_using_net_income21', 'roe_using_net_income21']
    df = tratar_outliers(df, vars_with_outliers)
    df[vars_with_outliers] = StandardScaler().fit_transform(df[vars_with_outliers])

    # Guardar el DataFrame transformado en un archivo CSV
    df.to_csv('data_3.csv', index=False)
    print("Archivo guardado como 'data_3.csv'")

    return df

In [84]:
transformar_dataframe(df)

Archivo guardado como 'data_3.csv'


,esg_score21,esg_class21,environ_score21,environ_class21,social_score21,social_class21,govern_score21,govern_class21,turnover21,country,...,esg_exe_remuneration,esg_respect_humanr,esg_human_rights,esg_respect_freedom,esg_non_discrimination,esg_physical_risk,esg_energy_transition,esg_carbon_emission1,esg_carbon_emission2,esg_carbon_emission1_2
0,-0.569247,Weak performance,-1.721908,Weak performance,-1.041523,Weak performance,0.656902,Limited performance,1.061682,USA,...,0.614529,-1.304142,-0.000852,-0.813810,0.151979,-0.761158,-1.539686,0.016030,0.054868,-0.777691
1,0.534502,Weak performance,0.446051,Weak performance,-0.150737,Weak performance,1.204951,Limited performance,1.361008,USA,...,2.139914,-0.113515,0.403974,-0.601699,0.686580,2.077778,0.207338,-0.530961,1.017824,0.489308
2,1.454293,Limited performance,0.686935,Weak performance,1.096364,Weak performance,2.301049,Limited performance,-0.059054,USA,...,2.444991,1.870863,1.213625,-0.389588,1.933982,2.077778,-0.195822,-1.018312,-0.001435,-0.585596
3,0.166586,Weak performance,-0.637929,Weak performance,-0.328894,Weak performance,0.930926,Limited performance,-0.582949,USA,...,0.716222,0.547944,0.269032,-0.601699,0.864780,-0.534043,-0.867754,0.067916,0.631736,0.148261
4,0.166586,Weak performance,0.084724,Weak performance,-0.685208,Weak performance,1.067939,Limited performance,-2.161638,USA,...,1.631453,0.018777,-0.000852,-1.662252,0.419279,-0.534043,-0.061435,-1.302281,-1.033073,-1.255900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22576,0.718460,Weak performance,0.927820,Weak performance,0.918207,Weak performance,1.204951,Limited performance,-0.000477,Italia,...,1.224683,0.680236,1.618450,1.731519,1.488481,1.737106,2.088748,0.747307,0.850895,0.581316
22577,0.350544,Weak performance,0.686935,Weak performance,-0.150737,Weak performance,0.793914,Limited performance,-0.444657,Italia,...,1.529760,1.341696,0.673857,-0.601699,1.310281,-0.761158,0.879270,-1.400355,-1.148276,-1.349291
22578,0.166586,Weak performance,0.927820,Weak performance,0.383735,Weak performance,0.656902,Limited performance,0.475488,Italia,...,1.834837,1.738571,1.618450,1.519408,2.379483,2.077778,1.819975,-0.898235,-0.944130,-1.087863
22579,0.718460,Weak performance,-0.637929,Weak performance,0.383735,Weak performance,0.930926,Limited performance,-0.683546,Italia,...,0.614529,-0.113515,0.538916,-0.389588,0.062878,-0.761158,0.341724,0.562537,-0.074874,0.054338


1. Variables en USD (Monetarias)
Estas variables tienen valores grandes y sesgo hacia la derecha.

Transformación: log1p (logarítmica) para comprimir la escala.
Normalización posterior: StandardScaler o MinMaxScaler.
Variables:

'turnover21', 'ebt21', 'net_income21', 'cash_flow21', 'total_assets21',
'equity21', 'gross_profit', 'liabilities21', 'ebitda21'

2. Variables de Puntaje (Scores ESG y Otros)
Estas variables son comparables, pero tienen rangos distintos.

Transformación: StandardScaler (Z-score), ya que preserva la distribución original pero normaliza las magnitudes.
Variables:

'esg_score21', 'environ_score21', 'social_score21', 'govern_score21',
'hrscore21', 'envscore21', 'businesscore21', 'governscore21', etc.

3. Variables Enteras (Firm Age, Employees, Board Members, etc.)
Estas variables tienen un rango amplio y valores dispersos.

Transformación: Log-transformación si hay dispersión alta o StandardScaler si tienen un rango manejable.
Variables:

'firm_age', 'employees21', 'board_members', 'investors'

4. Variables de Ratios y Márgenes
Estas variables suelen estar entre rangos manejables, pero es importante escalarlas uniformemente para compararlas.

Transformación: MinMaxScaler para llevarlas a un rango [0, 1].
Variables:

'margen_ebitda21', 'debt_ratio21', 'current_ratio21', 'solvency21',
'liquidity_ratio21', 'gross_margin21

## Tratamiento de las emisiones de dioxido de carbono 

Distribución altamente sesgada:

La media es mucho mayor que la mediana, lo cual indica un fuerte sesgo positivo (algunos valores extremadamente grandes – outliers).
El máximo es varias órdenes de magnitud superior al percentil 75.
Rango extremadamente amplio:

Ejemplo: esg_carbon_emission1 tiene un máximo de 336 millones y un mínimo de 0.
Presencia de valores pequeños y grandes:

El percentil 25 y el 50 son relativamente bajos comparados con el máximo.
No hay valores nulos: Todas las observaciones están completas.

# esg_size

Variables binarias que no requieren tratamiento: 'size_class', 'other_acount', 'legal_event', 'list_status'

group_size: Aplicar una transformación logarítmica para reducir el efecto del sesgo positivo.

cf_to_op_income21, roa_ebit21, roa_using_net_income21, roe_using_net_income21:

Tratamiento de outliers mediante recorte (winsorization) o filtrado usando IQR.
Opcionalmente, aplicar StandardScaler para normalizar estos valores.